# Conway's Reverse Game of Life

## Modified the metric to be `cell_acc` instead of `accuracy` due to top-3 prediction

# Keras - 1 Attempt

# Attempt 1

In [ ]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy,BinaryAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('conway-s-reverse-game-of-life/train.csv.zip')
test_df = pd.read_csv('conway-s-reverse-game-of-life/test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('conway-s-reverse-game-of-life/sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
from tensorflow.keras import layers, models
model = models.Sequential()

if n_samples > 10000 and n_features > 100:
    layer_sizes = [min(int(n_features * i), 1024) for i in (2, 1, 0.5, 0.25)]
    layer_sizes = [sz for sz in layer_sizes if sz >= 16]
    for sz in layer_sizes:
        model.add(layers.Dense(sz, activation='relu'))
        model.add(layers.BatchNormalization())
        model.add(layers.Dropout(0.3))
else:
    for sz in [min(n_features * 2, 128), min(n_features, 64)]:
        model.add(layers.Dense(sz, activation='relu'))
        model.add(layers.Dropout(0.3))

# Output layer
model.add(layers.Dense(output_units, activation='sigmoid'))

# Compile
model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=[BinaryAccuracy(name='cell_acc'), tf.keras.metrics.Precision(), tf.keras.metrics.Recall()]
)

# Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True),
    ModelCheckpoint('best_model.h5', save_best_only=True)
]

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_train,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
results = {
    'training_accuracy': history.history['cell_acc'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_cell_acc'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Epoch 1/100


313/313 - 8s - 25ms/step - cell_acc: 0.7655 - loss: 0.5049 - precision_1: 0.2181 - recall_1: 0.2383 - val_cell_acc: 0.8545 - val_loss: 0.3754 - val_precision_1: 0.3676 - val_recall_1: 0.0031
Epoch 2/100


313/313 - 4s - 14ms/step - cell_acc: 0.8533 - loss: 0.3707 - precision_1: 0.3324 - recall_1: 0.0108 - val_cell_acc: 0.8548 - val_loss: 0.3523 - val_precision_1: 0.5043 - val_recall_1: 0.0016
Epoch 3/100


313/313 - 4s - 14ms/step - cell_acc: 0.8543 - loss: 0.3572 - precision_1: 0.4158 - recall_1: 0.0101 - val_cell_acc: 0.8549 - val_loss: 0.3440 - val_precision_1: 0.5310 - val_recall_1: 0.0061
Epoch 4/100


313/313 - 4s - 12ms/step - cell_acc: 0.8543 - loss: 0.3509 - precision_1: 0.4400 - recall_1: 0.0157 - val_cell_acc: 0.8550 - val_loss: 0.3394 - val_precision_1: 0.5274 - val_recall_1: 0.0096
Epoch 5/100


...

313/313 - 5s - 15ms/step - cell_acc: 0.8554 - loss: 0.3163 - precision_1: 0.5156 - recall_1: 0.0619 - val_cell_acc: 0.8574 - val_loss: 0.3213 - val_precision_1: 0.6792 - val_recall_1: 0.0337
Epoch 79/100
313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3163 - precision_1: 0.5164 - recall_1: 0.0622 - val_cell_acc: 0.8575 - val_loss: 0.3214 - val_precision_1: 0.6836 - val_recall_1: 0.0338
Epoch 80/100


313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3163 - precision_1: 0.5174 - recall_1: 0.0621 - val_cell_acc: 0.8575 - val_loss: 0.3213 - val_precision_1: 0.6854 - val_recall_1: 0.0336
Epoch 81/100
313/313 - 5s - 16ms/step - cell_acc: 0.8555 - loss: 0.3161 - precision_1: 0.5174 - recall_1: 0.0618 - val_cell_acc: 0.8574 - val_loss: 0.3213 - val_precision_1: 0.6847 - val_recall_1: 0.0326
Epoch 82/100


313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3161 - precision_1: 0.5170 - recall_1: 0.0627 - val_cell_acc: 0.8574 - val_loss: 0.3213 - val_precision_1: 0.6837 - val_recall_1: 0.0324
Epoch 83/100


313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3160 - precision_1: 0.5168 - recall_1: 0.0628 - val_cell_acc: 0.8575 - val_loss: 0.3213 - val_precision_1: 0.6801 - val_recall_1: 0.0342
Epoch 84/100


313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3157 - precision_1: 0.5171 - recall_1: 0.0636 - val_cell_acc: 0.8574 - val_loss: 0.3212 - val_precision_1: 0.6783 - val_recall_1: 0.0342
Epoch 85/100
313/313 - 5s - 15ms/step - cell_acc: 0.8555 - loss: 0.3158 - precision_1: 0.5179 - recall_1: 0.0634 - val_cell_acc: 0.8574 - val_loss: 0.3213 - val_precision_1: 0.6848 - val_recall_1: 0.0332
Epoch 86/100
313/313 - 4s - 14ms/step - cell_acc: 0.8555 - loss: 0.3156 - precision_1: 0.5160 - recall_1: 0.0635 - val_cell_acc: 0.8575 - val_loss: 0.3213 - val_precision_1: 0.6754 - val_recall_1: 0.0358
Epoch 87/100
313/313 - 5s - 14ms/step - cell_acc: 0.8555 - loss: 0.3155 - precision_1: 0.5165 - recall_1: 0.0635 - val_cell_acc: 0.8574 - val_loss: 0.3213 - val_precision_1: 0.6819 - val_recall_1: 0.0339
Epoch 88/100
313/313 - 4s - 14ms/step - cell_acc: 0.8555 - loss: 0.3155 - precision_1: 0.5172 - recall_1: 0.0640 - val_cell_acc: 0.8575 - val_loss: 0.3213 - val_precision_1: 0.6829 - val_recall_1: 

# Conway's Reverse Game of Life

## Keras Tuner - 2 Attempts

## Attempt 1 

In [ ]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('conway-s-reverse-game-of-life/train.csv.zip')
test_df = pd.read_csv('conway-s-reverse-game-of-life/test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('conway-s-reverse-game-of-life/sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = X_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        output_units = len(target_columns)
        x = Dense(output_units, activation='sigmoid')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', 'Precision', 'Recall'])
        return model

# Tuner
bs = 64  # Example batch size
ep = 100  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=1,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=1,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=1, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

NameError: name 'X_val_proc' is not defined

## Attempt 2

In [ ]:
# Reproducibility
import random
import numpy as np
import pandas as pd
import json
import time
import tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import SparseTopKCategoricalAccuracy, BinaryAccuracy

# Set seeds for reproducibility
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

# Data Loading
train_df = pd.read_csv('conway-s-reverse-game-of-life/train.csv.zip')
test_df = pd.read_csv('conway-s-reverse-game-of-life/test.csv.zip')

# Infer id and target columns from sample submission
submission_example = pd.read_csv('conway-s-reverse-game-of-life/sampleSubmission.csv.zip', nrows=1)
cols = list(submission_example.columns)
id_col = cols[0]
target_columns = cols[1:]

# Combine training data (if multiple files)
df = train_df.copy()

def prepare_targets(df, target_columns, problem_subtype):
    if problem_subtype in ['multi-label classification']:
        mlb = MultiLabelBinarizer()
        return mlb.fit_transform(df[target_columns]), mlb.classes_
    elif problem_subtype in ['multiclass-classification', 'multiclass classification', 'ordinal-regression']:
        # Here we treat each cell state independently; use raw values
        return df[target_columns].values, None
    else:
        return df[target_columns].values, None

competition_problem_subtype = 'multiclass-classification'
y_train, classes_ = prepare_targets(df, target_columns, competition_problem_subtype)

# Feature matrix
X = df.drop(columns=[id_col] + target_columns, errors='ignore')

# Test features
X_test = test_df.drop(columns=[id_col] + target_columns, errors='ignore')
test_ids = test_df[id_col]

# Feature Engineering: drop all-missing columns
drop_cols = X.columns[X.isna().all()].tolist()
X.drop(columns=drop_cols, inplace=True)
X_test = X_test[X.columns]

# Identify numeric vs. categorical
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object', 'category']).columns.tolist()
# Remove high-cardinality categoricals
categorical_cols = [c for c in categorical_cols if X[c].nunique() <= 50]

# Preprocessing pipelines
numeric_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
cat_transformer = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer([
    ('num', numeric_transformer, numeric_cols),
    ('cat', cat_transformer, categorical_cols)
])

# Fit & transform
X_train_proc = preprocessor.fit_transform(X)
X_test_proc = preprocessor.transform(X_test)

# Split the data for validation
X_train_proc, X_val_proc, y_train, y_val = train_test_split(X_train_proc, y_train, test_size=0.2, random_state=seed)

# Model Architecture
n_samples, n_features = X_train_proc.shape
output_units = len(target_columns)
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Preserve input dim
n_features = X_train_proc.shape[1]

class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        output_units = len(target_columns)
        x = Dense(output_units, activation='sigmoid')(x)
        model = Model(inputs, x)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=[BinaryAccuracy(name='cell_acc'), 'Precision', 'Recall'])
        return model

# Tuner
bs = 64  # Example batch size
ep = 100  # Example epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=False,
    project_name='bayesian_tuner'
)

if y_val is not None:
    tuner.search(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_train,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain model with original callbacks and data
start_time = time.time()  # Start time for training
if y_val is not None:
    history = model.fit(
        X_train_proc, y_train,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_train,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
end_time = time.time()  # End time for training
print(f"Training time: {end_time - start_time} seconds")

# Logging results
results = {
    'training_accuracy': history.history['cell_acc'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_cell_acc'][-1],
    'validation_loss': history.history['val_loss'][-1]
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction & Submission
raw_preds = model.predict(X_test_proc)
final = (raw_preds > 0.5).astype(int)
if final.ndim == 1:
    final = final.reshape(-1, 1)

submission = pd.DataFrame(final, columns=target_columns)
submission.insert(0, id_col, test_ids.reset_index(drop=True))
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 06m 24s]
val_loss: 0.3264213502407074

Best val_loss So Far: 0.31926238536834717
Total elapsed time: 00h 47m 08s
Epoch 1/100


625/625 - 5s - 8ms/step - Precision: 0.3847 - Recall: 0.0312 - cell_acc: 0.8522 - loss: 0.3579 - val_Precision: 0.5539 - val_Recall: 0.0470 - val_cell_acc: 0.8561 - val_loss: 0.3294
Epoch 2/100


625/625 - 3s - 5ms/step - Precision: 0.5180 - Recall: 0.0648 - cell_acc: 0.8555 - loss: 0.3330 - val_Precision: 0.5676 - val_Recall: 0.0660 - val_cell_acc: 0.8571 - val_loss: 0.3240
Epoch 3/100


625/625 - 3s - 5ms/step - Precision: 0.5359 - Recall: 0.0742 - cell_acc: 0.8563 - loss: 0.3277 - val_Precision: 0.5782 - val_Recall: 0.0717 - val_cell_acc: 0.8576 - val_loss: 0.3221
Epoch 4/100


625/625 - 3s - 6ms/step - Precision: 0.5478 - Recall: 0.0786 - cell_acc: 0.8569 - loss: 0.3250 - val_Precision: 0.5833 - val_Recall: 0.0754 - val_cell_acc: 0.8579 - val_loss: 0.3209
Epoch 5/100


625/625 - 3s - 6ms/step - Precision: 0.5532 - Recall: 0.0817 - cell_acc: 0.8572 - loss: 0.3232 - val_Precision: 0.5876 - val_Recall: 0.0748 - val_cell_acc: 0.8580 - val_loss: 0.3202
Epoch 6/100


625/625 - 3s - 5ms/step - Precision: 0.5584 - Recall: 0.0853 - cell_acc: 0.8575 - loss: 0.3215 - val_Precision: 0.5870 - val_Recall: 0.0791 - val_cell_acc: 0.8582 - val_loss: 0.3198
Epoch 7/100


625/625 - 3s - 5ms/step - Precision: 0.5625 - Recall: 0.0876 - cell_acc: 0.8577 - loss: 0.3205 - val_Precision: 0.5973 - val_Recall: 0.0734 - val_cell_acc: 0.8583 - val_loss: 0.3194
Epoch 8/100
625/625 - 3s - 5ms/step - Precision: 0.5632 - Recall: 0.0893 - cell_acc: 0.8578 - loss: 0.3194 - val_Precision: 0.5934 - val_Recall: 0.0815 - val_cell_acc: 0.8585 - val_loss: 0.3195
Epoch 9/100


625/625 - 4s - 6ms/step - Precision: 0.5653 - Recall: 0.0911 - cell_acc: 0.8579 - loss: 0.3187 - val_Precision: 0.6002 - val_Recall: 0.0790 - val_cell_acc: 0.8586 - val_loss: 0.3192
Epoch 10/100
625/625 - 3s - 5ms/step - Precision: 0.5679 - Recall: 0.0931 - cell_acc: 0.8581 - loss: 0.3179 - val_Precision: 0.6010 - val_Recall: 0.0791 - val_cell_acc: 0.8587 - val_loss: 0.3193
Epoch 11/100
625/625 - 3s - 6ms/step - Precision: 0.5693 - Recall: 0.0944 - cell_acc: 0.8582 - loss: 0.3172 - val_Precision: 0.6046 - val_Recall: 0.0778 - val_cell_acc: 0.8587 - val_loss: 0.3194
Epoch 12/100


625/625 - 4s - 6ms/step - Precision: 0.5718 - Recall: 0.0959 - cell_acc: 0.8584 - loss: 0.3166 - val_Precision: 0.6045 - val_Recall: 0.0787 - val_cell_acc: 0.8588 - val_loss: 0.3192
Epoch 13/100
625/625 - 4s - 6ms/step - Precision: 0.5723 - Recall: 0.0965 - cell_acc: 0.8584 - loss: 0.3161 - val_Precision: 0.6044 - val_Recall: 0.0811 - val_cell_acc: 0.8589 - val_loss: 0.3194
Epoch 14/100
625/625 - 4s - 6ms/step - Precision: 0.5742 - Recall: 0.0975 - cell_acc: 0.8585 - loss: 0.3158 - val_Precision: 0.6021 - val_Recall: 0.0822 - val_cell_acc: 0.8589 - val_loss: 0.3195
Epoch 15/100
625/625 - 3s - 6ms/step - Precision: 0.5751 - Recall: 0.0988 - cell_acc: 0.8586 - loss: 0.3151 - val_Precision: 0.6087 - val_Recall: 0.0788 - val_cell_acc: 0.8589 - val_loss: 0.3193
Epoch 16/100
625/625 - 4s - 6ms/step - Precision: 0.5766 - Recall: 0.0990 - cell_acc: 0.8587 - loss: 0.3148 - val_Precision: 0.6052 - val_Recall: 0.0804 - val_cell_acc: 0.8589 - val_loss: 0.3193
Epoch 17/100
625/625 - 4s - 6ms/step -